# Контекстный менеджер
**Менеджер контекста** (context manager) — это некий объект, реализующий одноимённый протокол.

## Менеджер контекста для подключения к базе данных

In [19]:
from contextlib import contextmanager
import sqlite3
from typing import Generator


@contextmanager
def db_connection(db_path: str) -> Generator[sqlite3.Connection, None, None]:
    '''
    Context manager for safe database connection.
    '''
    conn = None
    try:
        conn = sqlite3.connect(db_path)
        yield conn
        conn.commit()
    except sqlite3.Error:
        if conn:
            conn.rollback()
        raise
    finally:
        if conn:
            conn.close()


with sqlite3.connect('test.db') as tmp_conn:
    tmp_conn.execute('CREATE TABLE IF NOT EXISTS users (id INTEGER, name TEXT)')


try:
    with db_connection('test.db') as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO users VALUES (1, "Алиса")')
        # raise sqlite3.OperationalError("Искусственная ошибка!")
except Exception as e:
    print(f'Error during database operation "{e}"')
else:
    print('Operation completed successfully!')

Error during database operation "database is locked"


## Менеджер контекста для операций с файлом

In [2]:
from contextlib import contextmanager
from typing import IO
from typing import Generator


@contextmanager
def open_file(path: str, mode: str) -> Generator[IO, None, None]:
    file = None
    try:
        file = open(path, mode)
        yield file
    finally:
        if file:
            file.close()



with open_file('test.txt', 'w') as file:
    file.write('Hello, Context Manager!')

## Асинхронный менеджер контекста

In [1]:
import aiohttp
from IPython.display import Markdown


class AsyncWeatherFetcher:
    '''
    Asynchronous context manager for getting weather via wttr.in
    '''
    
    def __init__(self, city: str = 'Пермь', format: str = '%C+%t+%w'):
        self.city = city
        self.format = format
        self.session = None
        self.response = None

    async def __aenter__(self):
        self.session = aiohttp.ClientSession()
        url = f'https://wttr.in/{self.city}?format={self.format}&lang=ru'
        self.response = await self.session.get(url)
        return self

    async def __aexit__(self, exc_type, exc, tb):
        if self.response:
            self.response.close()
        if self.session:
            await self.session.close()
        if exc_type:
            print(f'⚠️ Error: {exc}')

    async def get_weather(self) -> str:
        '''
        Returns a formatted string with the weather
        '''
        if not self.response:
            raise RuntimeError('Log in to the context manager!')
        return await self.response.text()


async def main():
    async with AsyncWeatherFetcher('Paris', '%C+%t+%w+%h') as fetcher:
        weather = await fetcher.get_weather()
        display(Markdown(f'## 🌍 Weather in Paris: {weather}'))


await main()

## 🌍 Weather in Paris: Солнечно +25°C ↓11km/h 32%

In [5]:
from contextlib import asynccontextmanager
import aiohttp


@asynccontextmanager
async def async_http_client(url: str):
    '''
    Asynchronous context manager for HTTP sessions.
    '''
    session = aiohttp.ClientSession()
    try:
        print(f'🔹 Opening session for {url}')
        yield session
        print('🔹 Session finished successfully')
    except Exception as e:
        print(f'❌ Session error: {e}')
        raise
    finally:
        await session.close()
        print('🔹 Session has been closed')


async def fetch_data():
    async with async_http_client('https://api.example.com') as client:
        async with client.get('/data') as response:
            return await response.json()


# for .py
# asyncio.run(fetch_data())
# for .ipynb
await fetch_data()

🔹 Opening session for https://api.example.com
❌ Session error: /data
🔹 Session has been closed


InvalidUrlClientError: /data